In [1]:
import torch
from torch_geometric.nn import LightGCN
from torch_geometric.datasets import MovieLens100K, AmazonBook, Yelp
import os.path as osp

In [3]:
import sys
# Add the path to the "src" directory where your models are located
sys.path.append('/Users/hania/Desktop/inzynierka/recommender-systems-gnn/src')
from models.lightgcn import LightGCN


ModuleNotFoundError: No module named 'src'

In [2]:
path = '../../data/raw/ml-100k'
dataset = MovieLens100K(path)
data = dataset[0]
print(data)

Processing...


HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


Done!


In [21]:
# to make user index before movie index
data.rename('movie', 'Movie')

HeteroData(
  user={ x=[943, 24] },
  Movie={ x=[1682, 18] },
  (user, rates, Movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (Movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)

In [22]:
data

HeteroData(
  user={ x=[943, 24] },
  Movie={ x=[1682, 18] },
  (user, rates, Movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (Movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)

In [5]:
data = data.to_homogeneous()

In [6]:
data.edge_type

tensor([0, 0, 0,  ..., 1, 1, 1])

In [31]:
data.x[1499]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [7]:
# Use all message passing edges as training labels:
batch_size = 81920
# remove duplicate edges
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

In [8]:
print(train_edge_label_index)

tensor([[   0,    0,    0,  ...,  942,  942,  942],
        [ 943,  944,  945,  ..., 2130, 2170, 2272]])


In [9]:
# model = LightGCN(
#     num_nodes=data.num_nodes,
#     embedding_dim=64,
#     num_layers=2,
# )
model = NGCF(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2,
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
from tqdm import tqdm

def train():
    total_loss = total_examples = 0

    for index in tqdm(train_loader):
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_movie,
                          (index.numel(), ))
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

In [12]:
from torch_geometric.utils import degree

@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, movie_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ movie_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [13]:
for epoch in range(1, 10):
    loss = train()
    precision, recall = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f}')

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.31s/it]


Epoch: 001, Loss: 0.6931, Precision@20: 0.0435, Recall@20: 0.0198


100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


Epoch: 002, Loss: 0.6931, Precision@20: 0.0878, Recall@20: 0.0377


...

Epoch: 007, Loss: 0.6904, Precision@20: 0.2254, Recall@20: 0.1497


100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


Epoch: 008, Loss: 0.6892, Precision@20: 0.2349, Recall@20: 0.1555


100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


Epoch: 009, Loss: 0.6877, Precision@20: 0.2373, Recall@20: 0.1553
